# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выведем первые 15 строчек для общего представления о данных**

In [4]:
display (data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем последние 15 строчек для общего представления о данных**

In [5]:
display (data.tail(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


### Описание данных
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

### Анализ данных
Из общей инофрмации по таблице мы видим что столбцы `days_employed` и  `total_income` имеют одинаковое кол-во пропусков.
Нужно проверить: В одних и тех же строках отсутствуют данные по столбцам `days_employed` и `total_income`

In [6]:
data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Да, действительно, данные отсутсвуют в одних и тех же строках. Теперь необходимо проверить их тип деятельности `income_type`, мало ли они все одной профессии и их можно будет заполнить медианным значением по аналогичному роду деятельности.

In [7]:
data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Нет, теория не подтвердилась, следовательно придется заполнять пропущеные строки по медианным значениям разных типов занятости

Применим метод `describe()` чтобы удостовериться в первоначальных предположениях

In [8]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Тут к нашим наблюдениям добавляется следующее:
* минимальное значение столбца `children` равно -1, этого не может быть
* максимальное значение столбца `children` равно 20, это вполне реально, но это прямо влияет на решение поставленной задачи, возможно это аномалия? Целых 75% имеет только 1 ребенка, нужно проверить есть ли еще такие аномальыне значения.
* возраст по некоторым строкам `dob_years` равен 0
* среднее значение в столбце `days_employed` 63046 дней, а это ~ 173 года, вот из за них пенсионный возраст и повысили)
* максимальные значение в столбце `days_employed` 401755 дней, а это ~ 1 100 лет это явно ошибка

### Выводы
* 1) Мы видим что столбцы `days_employed` и  `total_income` имеют пропуски, возможно клиенты не указаыали эту информацию или же просто банк не запрашивал ее. Скорее всего придется заполнять его аналогично с другими строками по столбцу `income_type`
* 2) В столбце `days_employed` присутсвуют отрицательные значения, скорее всего это ошибка при записи данных, так как трудовой стаж не может быть отрицательным, также в некоторых строчках присутствуют нереальные значения. Тут скорее всего нужно использовать значение по модулю
* 3) В столбце `education` присутсвуют как строчные, так и прописные буквы, их нужно будет привести в единый формат, тут вероятнее всего не было требований к регистру при заполнения данных, поэтому данные так и записывались
* 4) В столбце `children` есть ошибка в данных и возможно аномалия, нужно будет это првоерить
* 5) В столбце `dob_years` есть ошибка в данных, ее необходимо исправить

## Шаг 2. Предобработка данных

### Стиль заголовков


Проверим название столбцов на наличие ошибок

In [9]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Ошибок не обнаружено! Дальше для наглядности подсчиатем сумму пропущенных значений

### Регистр даннных
В столбце `education` присутсвуют как строчные, так и прописные буквы, их нужно будет привести в единый вид. Для этого нужно использовать метод `str.lower()`

In [10]:
data['education'] = data['education'].str.lower()

Проверим изменения

In [11]:
display (data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Замена отрицательных значений
Для наглядности выведем количество пропусков еще раз. Далее создадим `pivot_table` с группировкой по `income_type`, чтобы посмотреть из за чего происходит такой "перекос" по данным, где данные отрицательные, а где с данными все в порядке.

In [12]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [13]:
data_grouped = data.groupby('income_type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]}) #создаем новую таблицу
#с группировкой по столбцу 'income_type', также применем к ней метод agg()
data_grouped = data_grouped.rename(columns={'count': 'Кол-во строк', 'mean': 'Среднее кол-во рабочих дней'})# переименуем название столбцов
display (data_grouped)

days_employed                                       
                 Кол-во строк Среднее кол-во рабочих дней <lambda_0>
income_type                                                         
безработный                 2               366413.652744        2.0
в декрете                   1                -3296.759962        0.0
госслужащий              1312                -3399.896902        0.0
компаньон                4577                -2111.524398        0.0
пенсионер                3443               365003.491245     3443.0
предприниматель             1                 -520.848083        0.0
сотрудник               10014                -2326.499216        0.0
студент                     1                 -578.751554        0.0

Исходя из датафрейма можно сделать такие выводы: 
* 1) По безработным и пенсионерам зачения положительные
* 2) По остальным наоборот все значения отрицательные
* 3) Переведем к абсолютному значению единстенный отрицательное значение по `children`

Следовательно мы сможем применить `abs()` - абсолютное значение к этим столбцам

In [14]:
data[['days_employed', 'total_income', 'children']] = data[['days_employed', 'total_income', 'children']].abs()

### Замена пропусков и аномальных значений
В одной из строчек `children` есть значение 20, которое выбивается из общей картины, в виду того, что неизвестно ошибка это ввода данных (написали лишний 0 в конце) или же ошибка записи данных, поменяем ее на медианое значение.
В столбцах `days_employed` и `total_income` мы обнаружили пропущенные значения в одинаковых строчках, произведем замену на средний опыт работы и медианный доход



In [15]:
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median()
data['children'] = data['children'].replace(20, children_median)
total_income_median = data.groupby('income_type')['total_income'].median()
round(total_income_median)

income_type
безработный        131340.0
в декрете           53829.0
госслужащий        150448.0
компаньон          172358.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98202.0
Name: total_income, dtype: float64

In [16]:
# Заменим пропущенные значения ежемесячного дохода на медианы по типам занятости.
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'сотрудник'), 'total_income'] = total_income_median[6]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'компаньон'), 'total_income'] = total_income_median[3]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'пенсионер'), 'total_income'] = total_income_median[4]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'госслужащий'), 'total_income'] = total_income_median[2]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'предприниматель'), 'total_income'] = total_income_median[5]

In [17]:
#Создадим возрастную категорию
age_stat = data['dob_years'].describe()
def age_group(age):
    if age <= age_stat[4]: return 'до 34'
    elif age_stat[4] < age <= age_stat[5]: return '34-43'
    elif age_stat[5] < age <= age_stat[6]: return '43-53'
    else: return '53+'
    
data['age_group'] = data['dob_years'].apply(age_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1.0,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,34-43
1,1.0,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,34-43
2,0.0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,до 34
3,3.0,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,до 34
4,0.0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,43-53


In [18]:
days_employed_median = data.groupby('age_group')['days_employed'].median()
round(days_employed_median, 0)

age_group
34-43      1812.0
43-53      2441.0
53+      342859.0
до 34      1147.0
Name: days_employed, dtype: float64

In [19]:
# Заменим пропущенные значения трудового стажа на медиану из возрастных категорий
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == '34-43'), 'days_employed'] = days_employed_median[0]
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == '43-53'), 'days_employed'] = days_employed_median[1]
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == '53+'), 'days_employed'] = days_employed_median[2]
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == 'до 34'), 'days_employed'] = days_employed_median[3]

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null float64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


Все пропуски устранены

### Замена типа данных
В столбцах `days_employed` и `total_income` данные в формате float, заменим их на int для удобства (целочисленные значения легче воспринимать)

In [21]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов
Проверим сумарное колличество дубликатов для этого используем стандартный метод `duplicated()`

In [22]:
data.duplicated().sum()

71

Для наглядности выведем первые 5 строк дубликатов на экран

In [23]:
data[data.duplicated(keep = False)].sort_values('dob_years', ascending = False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
7938,0.0,342859,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
9604,0.0,342859,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
6537,0.0,342859,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
5865,0.0,342859,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью,53+
9528,0.0,342859,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью,53+


Удалим дубликаты и проверим их сумму

In [24]:
data = data.drop_duplicates()
data.duplicated().sum()

0

**Вывод**

Убрали дубликаты из датафрейма

### Лемматизация
В датафрейме есть столбец `purpose`, в нем содержатся похожие друг на друга цели кредита, но выраженные разными словами.
Надо подсчитать уникальные леммы и добавить новым столбцом

In [25]:
unique_purposes = data['purpose'].value_counts().index.tolist()
display (unique_purposes)

['свадьба',
 'на проведение свадьбы',
 'сыграть свадьбу',
 'операции с недвижимостью',
 'покупка коммерческой недвижимости',
 'операции с жильем',
 'покупка жилья для сдачи',
 'операции с коммерческой недвижимостью',
 'покупка жилья',
 'жилье',
 'покупка жилья для семьи',
 'строительство собственной недвижимости',
 'недвижимость',
 'операции со своей недвижимостью',
 'строительство жилой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'строительство недвижимости',
 'ремонт жилью',
 'покупка жилой недвижимости',
 'на покупку своего автомобиля',
 'заняться высшим образованием',
 'автомобиль',
 'сделка с подержанным автомобилем',
 'автомобили',
 'свой автомобиль',
 'на покупку подержанного автомобиля',
 'на покупку автомобиля',
 'приобретение автомобиля',
 'дополнительное образование',
 'сделка с автомобилем',
 'высшее образование',
 'образование',
 'получение дополнительного образования',
 'получение образования',
 'профильное образование',
 'получение высшего образован

In [26]:
# Импортируем библиотеку *pymystem3* и *collections*
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

Исходя из списка целей можно выделить 5 категорий. Далее проведем лемматизацию и одновременно заменим в каждой строке значение на ключевое слово из категорий c помощью функции

In [27]:
categories = ["сдача", "коммерческий", "жилье", "образование", "свадьба", "недвижимость", "автомобиль"]
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma

data['purpose_group'] = data['purpose'].apply(lemmatize)        
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group
0,1.0,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,жилье
1,1.0,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,автомобиль
2,0.0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 34,жилье
3,3.0,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 34,образование
4,0.0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,43-53,свадьба


In [28]:
data['purpose_group'].value_counts() #подсчитаем категории целей

недвижимость    5040
автомобиль      4306
образование     4013
жилье           3809
свадьба         2324
коммерческий    1311
сдача            651
Name: purpose_group, dtype: int64

Дальше произведем объединение некоторых групп, а именно:
* жилье добавим к недвижимость
* сдача добавим к коммерческий

In [29]:
data.loc[data['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
data.loc[data['purpose_group'] == 'сдача', 'purpose_group'] = 'коммерческий'
data['purpose_group'].value_counts()

недвижимость    8849
автомобиль      4306
образование     4013
свадьба         2324
коммерческий    1962
Name: purpose_group, dtype: int64

**Вывод**
После того как мы лемматизировали данные у нас получилось выделить 5 групп:
* недвижимость
* автомобиль
* образование
* свадьба
* коммерческий

Данная процедура упрощает анализ целей кредита, а также в дальнейшем позволяет сгруппировать их по категориям

### Категоризация данных
Для того чтобы ответить на поставленные вопросы нам нужно создать 3 категории:
* возрастную
* по целям
* По доходам

In [30]:
income_stat = data['total_income'].describe()
income_stat[4:7]

25%    107623.00
50%    142594.00
75%    195820.25
Name: total_income, dtype: float64

Для категорий будем использовать (до 25%, от 25% до 50%, от 50% до 75%, больше 75%)

Напишем для этого функцию соответственно

In [31]:
# функция для определения категории доходов
def determine_income_group(income):
    if income <= income_stat[4]: return 1
    elif income_stat[4] < income <= income_stat[5]: return 2
    elif income_stat[5] < income <= income_stat[6]: return 3
    else: return 4

# применим функцию к столбцу доходов
data['income_group'] = data['total_income'].apply(determine_income_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,income_group
0,1.0,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,недвижимость,4
1,1.0,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,автомобиль,2
2,0.0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 34,недвижимость,3
3,3.0,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 34,образование,4
4,0.0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,43-53,свадьба,3


In [32]:
# напишем функцию, которая принимает на вход возраст клиента и возвращает возрастную категорию
def age_group(age):
    if age <= age_stat[4]: return 'до 34'
    elif age_stat[4] < age <= age_stat[5]: return '34-43'
    elif age_stat[5] < age <= age_stat[6]: return '43-53'
    else: return '53+'
    
data['age_group'] = data['dob_years'].apply(age_group)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,income_group
0,1.0,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,недвижимость,4
1,1.0,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,автомобиль,2
2,0.0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 34,недвижимость,3
3,3.0,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 34,образование,4
4,0.0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,43-53,свадьба,3


In [33]:
# Найдем медианное значение трудового стажа для каждой возрастной группы.
medians_of_days_employed = data.groupby('age_group')['days_employed'].median()
round(medians_of_days_employed, 0)

age_group
34-43      1811
43-53      2440
53+      342859
до 34      1146
Name: days_employed, dtype: int64

#### Категоризация по возрасту
Выведем результат категоризации по возрасту

In [34]:
data['age_group'].value_counts()

до 34    5467
43-53    5448
34-43    5284
53+      5255
Name: age_group, dtype: int64

#### Категоризация по целям

In [35]:
data['purpose_group'].value_counts()

недвижимость    8849
автомобиль      4306
образование     4013
свадьба         2324
коммерческий    1962
Name: purpose_group, dtype: int64

#### Категоризация по доходам

In [36]:
data['income_group'].value_counts()

2    5479
4    5364
1    5364
3    5247
Name: income_group, dtype: int64

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
print(data.groupby('children')['debt'].value_counts())

children  debt
0.0       0       13096
          1        1071
1.0       0        4410
          1         445
2.0       0        1858
          1         194
3.0       0         303
          1          27
4.0       0          37
          1           4
5.0       0           9
Name: debt, dtype: int64


Из данной таблицы сложно увидеть взаимосвязь между наличием детей и просрочкой выплаты, поэтому напишем функцию с применением категоризации по параметрам: наличие детей, колличество детей и наличие задолжности

In [38]:
def children_and_debt(row):
    children = row['children']
    debt = row['debt']
    
    if children == 0:
        return 'Без детей'
    
    if children >= 1:
        return 'С детьми'

In [39]:
data['children_return'] = data.apply(children_and_debt, axis=1)
print(data['children_return'].value_counts())

Без детей    14167
С детьми      7287
Name: children_return, dtype: int64


In [40]:
def relation(category):
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

In [41]:
relation('children_return')

,debt
children_return,
Без детей,0.075598
С детьми,0.091945


In [42]:
children_return_pivot = data.pivot_table(index='children_return', values='debt')
print(children_return_pivot)#вариант с применением pivot_tabl

                     debt
children_return          
Без детей        0.075598
С детьми         0.091945


**Вывод**
Исходя из анализа можно сделать вывод, что наличие детей негативно сказывается на своевременную оплату задолжностей, скорее всего это связано тем, что дети оказывают довольно серьезную материальную нагрузку на родителей

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для этого также напишем функцию по соответствующим параметрам

In [43]:
data.groupby(['family_status', 'family_status_id']).size()

family_status          family_status_id
Не женат / не замужем  4                    2810
в разводе              3                    1195
вдовец / вдова         2                     959
гражданский брак       1                    4151
женат / замужем        0                   12339
dtype: int64

Исходя из датафрейма семейный статус отмечается цифрой от 0 до 4, где:
* 0 - женат / замужем
* 1 - гражданский брак
* 2 - вдовец / вдова
* 3 - в разводе
* 4 - Не женат / не замужем	

In [44]:
def family_status_and_debt(row):
    family_status = row['family_status_id']
    debt = row['debt']
 
    if family_status == 0:
        return 'В браке'
               
    if family_status == 1:
        return 'В гражданском браке'
                
    if family_status == 2:
        return 'Вдовец / вдова'
             
    if family_status == 3:
        return 'В разводе'
                
    if family_status == 4 :
        return 'Не женат'

In [45]:
data['family_status_return'] = data.apply(family_status_and_debt, axis=1)
print(data['family_status_return'].value_counts())

В браке                12339
В гражданском браке     4151
Не женат                2810
В разводе               1195
Вдовец / вдова           959
Name: family_status_return, dtype: int64


In [46]:
relation('family_status_return')

,debt
family_status_return,
Вдовец / вдова,0.065693
В разводе,0.071130
В браке,0.075452
В гражданском браке,0.093471
Не женат,0.097509


In [47]:
family_status_pivot = data.pivot_table(index='family_status_return', values='debt')
print(family_status_pivot)#вариант с применением pivot_table

                          debt
family_status_return          
В браке               0.075452
В гражданском браке   0.093471
В разводе             0.071130
Вдовец / вдова        0.065693
Не женат              0.097509


**Вывод**
Можно сделать вывод, что не женатые люди более склонны к просрочке выплат, а что самое интересное вдовы/вдовцы менее. Возможно первое связано с тем, что люди еще не нашли себя в жизни и не имеют достаточных средств для выплат, а вторые получили имущество или пр. и уже имеют наработанный материальный запас

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Соответственно напишем функцию для того чтобы выделить категории клиентов по видам дохода

In [48]:
def total_income_and_debt(row):
    total_income = row['total_income']
    debt = row['debt']
#Низкий достаток:   
    if 0 < total_income <= 50000 :
        return 'С низким достатком'
        
# Средний достаток:        
    if 50000  < total_income <= 150000 :
        return 'С средним достатком'
        
# Высокий достаток:        
    if total_income > 150000:
        return 'С высоким достатком'

In [49]:
data['total_income_return'] = data.apply(total_income_and_debt, axis=1)
print(data['total_income_return'].value_counts())

С средним достатком    11251
С высоким достатком     9831
С низким достатком       372
Name: total_income_return, dtype: int64


In [50]:
relation('total_income_return')

,debt
total_income_return,
С низким достатком,0.061828
С высоким достатком,0.077612
С средним достатком,0.084881


In [51]:
total_income_pivot = data.pivot_table(index='total_income_return', values='debt')
print(total_income_pivot)#вариант с применением pivot_table

                         debt
total_income_return          
С высоким достатком  0.077612
С низким достатком   0.061828
С средним достатком  0.084881


**Вывод** Исходя из результатов можно сделать несколько выводов. Да, действительно чем выше уровень заработка, тем меньше вероятности того, что клиент не сможет во время погасить задолжность. **Но! Мы также видим, что высокий уровень заработка тоже не является весомым аргументом в своевременном погашении задожностей! (завышение уровня заработка, банкротство и пр.)**

Как итог самым оптимальным клиентом является клиент с средним заработком

- Как разные цели кредита влияют на его возврат в срок?

In [52]:
data['purpose_group'].value_counts()

недвижимость    8849
автомобиль      4306
образование     4013
свадьба         2324
коммерческий    1962
Name: purpose_group, dtype: int64

In [53]:
relation('purpose_group')

,debt
purpose_group,
недвижимость,0.071307
коммерческий,0.076962
свадьба,0.080034
образование,0.092200
автомобиль,0.093590


**Вывод**

Наименьшая склонность к просрочке  в недвижимости и коммерческой деятельности, наибольшая - в автомобилях. Возможно это связано с тем, что при приобретении автомобиля люди менее трепетно относятся к выплатам во время и это является их первым кредитом.

## Шаг 4. Общий вывод
Перед нами была поставлена задача ответить на вопросы:
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Проведя анализ мы можем ответь на них:
 1. Безусловно! Наличие детей негативно влияет на выплаты в срок. Как правило наличие детей обременяет семью.
 2. Да! Самые "неплательщики" это люди не сосоящие в браке, по середине люди в браке, а самые добропорядочне плательщики это вдовы/вдовцы. Первые возможно не имеют еще почвы под ногами, вторые могут иметь как и положительные факторы (семейный бюджет), так и негативные (дети, неработающий член семьи), а вот вдовы/вдовцы кроме того, что скорее всего уже имеют и большой стаж работы, так и еще возможно полуили часть наследства.
 3. Да! Люди с низким достатком чаще имеют задолжность в виду отсутствия средств. Люди со средним достатком совершают меньше просрочек чем с низким и высоким достатком. Однако люди с высоким чаще имеют задолжность, чем люди со средним достатком, скорее всего это связано с завышением доходов для получения более крупного кредита
 4. Да! Самая "неопасная цель" получеия кредита - это нежвижимость и коммерчиская. "опасная" - авто. Тут можно предположить то, что разница в объёмах кредита на эти цели очень большая, а также подготовленность людей к данным целям (первоначальный взнос, стартовый капитал, наличие другого бизнеса, подушка безопасности и др.) 
 Таким образом мы выяснили, что все 4 фактора оказывают зависимость к возврату долга в банке. 

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.